In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Generate Data from A Study of Sea kayak incidents in Ireland

### Field experience

.From [[1](https://issuu.com/lastudio/docs/14752_tnadt_63_issuu)] 70% of those in the survey had been paddling  5 or more years , with nearly half of them having 10 or more experience .  Therefore I generated a data set as below.
35% - five or more experience
35% - ten or more experince 
30% less than five years experience 

In [2]:
experienced = []
for i in range(35):
    experienced.append(np.random.randint(6))
for i in range(35):
    experienced.append(np.random.randint(11))
for i in range(30):
    experienced.append(np.random.randint(20))  
np.random.shuffle(experienced)

Years_Of_Experience = pd.Series(experienced)
Years_Of_Experience

0      1
1      1
2      0
3      2
4      1
      ..
95    19
96     2
97     1
98     1
99     5
Length: 100, dtype: int64

### Field Location



There was no information on location other than most incdients happened on the west coast. Thus therefore I split the 4 regions equally. 
%25 - west coast 
%25 east Coast
%25 South Coast
%25 North Coast



In [3]:

location = []
for i in range(25):
    location.append(0)
for i in range(25):
    location.append(1)
for i in range(25):
    location.append(2)
for i in range(25):
    location.append(3)    
np.random.shuffle(location)

for i in range(len(location)):
    if location[i] == 0:
        location[i] = "West Coast"
    elif location  == 1:
        location[i] = "East Coast"
    elif location  == 2:
        location[i] = "North Coast"
    else:
        location[i] = "South Coast"
location_of_incident = pd.Series(location)
location_of_incident

0      West Coast
1     South Coast
2     South Coast
3     South Coast
4     South Coast
         ...     
95    South Coast
96    South Coast
97     West Coast
98     West Coast
99    South Coast
Length: 100, dtype: object

### Field Wind Speed 

In [4]:
windSpeed = []
# 3rd of incidents happend in force 3 between 12km/h and 20 km/h
for i in range(33):
    windSpeed.append(np.random.randint(12,20))
#  80% of incidents happend in force 4 and under 
for i in range(47):
    windSpeed.append(np.random.randint(0,29))
# 20% of incidents happened above force 4 
for i in range(20):
    windSpeed.append(np.random.randint(29,40))  
np.random.shuffle(windSpeed)
Wind_Speed = pd.Series(windSpeed)
Wind_Speed

0     19
1     12
2     13
3      5
4     21
      ..
95    19
96     9
97    15
98    31
99    16
Length: 100, dtype: int64

### Group Size 

In [5]:
groupSize = []
# Most incidents occured when group size was between 7 and 8 people
for i in range(60):
    groupSize.append(np.random.randint(7,9))
#  Followed by between 3 and 4 people
for i in range(30):
    groupSize.append(np.random.randint(3,5))
# Only 4 incidents 1 person was in the paddle
for i in range(4):
    groupSize.append(1)
# All other size groups
groupSize.append(2)
groupSize.append(5)
groupSize.append(6)
groupSize.append(12)
groupSize.append(9)
groupSize.append(10)



np.random.shuffle(groupSize)
group_Size = pd.Series(groupSize)
group_Size

0      4
1      7
2      8
3      3
4      3
      ..
95     1
96    10
97     7
98     7
99     3
Length: 100, dtype: int64

### Field Formal Training 

In [26]:
training = []
for i in range(30):
    training.append(0)
for i in range(70):
    training.append(1)
 
np.random.shuffle(training)

for i in range(len(training)):
    if training[i] == 0:
        training[i] = "No"
    elif training[i]  == 1:
        training[i] = "Yes"
    
Formal_training = pd.Series(training)
Formal_training 

0      No
1     Yes
2     Yes
3     Yes
4     Yes
     ... 
95    Yes
96    Yes
97     No
98    Yes
99     No
Length: 100, dtype: object

### Field trip Type


In [22]:
tripType = []
for i in range(60):
    tripType.append(0)
for i in range(30):
    tripType.append(1)
for i in range(10):
    tripType.append(2)
 
np.random.shuffle(tripType)

for i in range(len(tripType)):
    if tripType[i] == 0:
        tripType[i] = "Cliff CoastLine"
    elif tripType[i]  == 1:
        tripType[i] = "Dumping Surf"
    else:
        tripType[i] = "Other"
        
    
Trip_Type = pd.Series(tripType)
Trip_Type

0     Cliff CoastLine
1     Cliff CoastLine
2     Cliff CoastLine
3        Dumping Surf
4     Cliff CoastLine
           ...       
95       Dumping Surf
96    Cliff CoastLine
97    Cliff CoastLine
98    Cliff CoastLine
99       Dumping Surf
Length: 100, dtype: object

In [27]:
frame = { 'experience': Years_Of_Experience, 'location': location_of_incident ,'wind Speed':Wind_Speed,'group Size':group_Size,
        'training':Formal_training,'trip Type':Trip_Type} 
  
result = pd.DataFrame(frame) 
result.describe()

,experience,wind Speed,group Size
count,100.000000,100.00000,100.000000
mean,5.350000,18.06000,5.950000
std,5.263511,9.76783,2.244522
min,0.000000,0.00000,1.000000
25%,1.000000,12.75000,4.000000
50%,4.000000,17.00000,7.000000
75%,8.000000,25.00000,7.250000
max,19.000000,39.00000,12.000000


In [29]:
result


,experience,location,wind Speed,group Size,training,trip Type
0,1,West Coast,19,4,No,Cliff CoastLine
1,1,South Coast,12,7,Yes,Cliff CoastLine
2,0,South Coast,13,8,Yes,Cliff CoastLine
3,2,South Coast,5,3,Yes,Dumping Surf
4,1,South Coast,21,3,Yes,Cliff CoastLine
...,...,...,...,...,...,...
95,19,South Coast,19,1,Yes,Dumping Surf
96,2,South Coast,9,10,Yes,Cliff CoastLine
97,1,West Coast,15,7,No,Cliff CoastLine
98,1,West Coast,31,7,Yes,Cliff CoastLine


### References 

1. Ray McCullagh and Keven 0'Callaghan,A Study of sea kayak incidents in Ireland https://issuu.com/lastudio/docs/14752_tnadt_63_issuu